## Enunciado

Considere o problema descrito no documento **+Lógica Computacional: Multiplicação de Inteiros**. Nesse documento, usa-se um “Control Flow Automaton” como modelo do programa imperativo que calcula a multiplicação de inteiros positivos representados por vetores de bits.

Pretende-se:

1. **Construir um SFOTS**, usando `BitVec` de tamanho \( n \), que descreva o comportamento deste autômato. Para isso:
   - Identifique e codifique em `Z3` ou `pySMT` as variáveis do modelo, o estado inicial, a relação de transição e o estado de erro.

2. **Usar \( k \)-indução** para verificar no SFOTS se a propriedade \( (x \times y + z = a \times b) \) é um invariante do seu comportamento.

3. **Verificar a segurança do programa**, usando \( k \)-indução no FOTS acima e adicionando ao estado inicial a condição \( (a < 2^{n/2}) \land (b < 2^{n/2}) \). Prove que, com tal estado inicial, o estado de erro nunca é acessível.



## Resolução

**Safe First Order Transition Systems (SFOTS)** são modelos de sistemas dinâmicos definidos por
$\:\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:\:$
onde 
$\,\mathsf{X}\,$ é o conjunto das variáveis base que define o espaço de estados,
$\,\mathsf{next}\,$ é o operador que gera os vários “clones” dessas variáveis,
$\;\mathsf{I}\;$ é o predicado que determina os estados iniciais,
$\;\mathsf{T}\;$ é o predicado que define a relação de transição entre estados, e
$\;\mathsf{E}\;$ é o predicado que define uma condição de erro.

In [1]:
from pysmt.shortcuts import Symbol, And, Or, Not, Equals, BV, BVAdd, BVUlt, BVShl, BVUrem, Ite
from pysmt.typing import BVType

import random

# Gera um número inteiro aleatório entre 1 e 100(como exemplo)
n = random.randint(1, 100)

# Variaveis de modelo
a = Symbol("a", BVType(n))
b = Symbol("b", BVType(n))
x = Symbol("x", BVType(n))
y = Symbol("y", BVType(n))
z = Symbol("z", BVType(n))

# Variáveis de estado (o dicionário `s`)
def init1(s):
    return And(Equals(s['x'], s['a']), Equals(s['y'], s['b']), Equals(s['z'], BV(0, n)))

def error1(s):
    overflow_shift = BVUlt(BV(2**n - 1, n), BVShl(s['x'], BV(1, n)))  # x << 1 excede o limite
    overflow_sum = BVUlt(BV(2**n - 1, n), BVAdd(s['z'], s['x']))      # z + x excede o limite
    return Or(overflow_shift, overflow_sum)

def trans1(curr, prox):
    
    #A transição simula uma etapa da multiplicação russa.
    even_y = Equals(BVUrem(curr['y'], BV(2, n)), BV(0, n))  # Verifica se y é par
    # Definindo as variáveis de próximo estado
    next_x = Ite(even_y, BVShl(curr['x'], BV(1, n)), curr['x'])          # Se y for par, x << 1
    next_y = Ite(even_y, curr['y'].BVLShr(BV(1, n)), BVAdd(curr['y'], BV(-1, n)))  # Se y for par, y >> 1; senão, y - 1
    next_z = Ite(even_y, curr['z'], BVAdd(curr['z'], curr['x']))         # Se y for ímpar, z + x

    # Definindo o próximo estado
    return And(Equals(prox['x'], next_x),Equals(prox['y'], next_y),Equals(prox['z'], next_z),Equals(prox['a'], curr['a']), Equals(prox['b'], curr['b']))


ImportError: cannot import name 'BVUlt' from 'pysmt.shortcuts' (C:\Users\HP\anaconda3\envs\logica\lib\site-packages\pysmt\shortcuts.py)